In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score

### 1. initialize

In [3]:
# data_src_path = './data/train.csv'
data_src_path = '../../data/original/train.csv'

data = pd.read_csv(data_src_path)
X = data.iloc[:, 2:].values
y = data['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.995, random_state=42)

### 2. train and predict

In [4]:
# train
clf = svm.SVC(
    C=1.0, 
    kernel='rbf', 
    degree=3, 
    gamma='scale'
)
clf.fit(X_train, y_train)

# predict
y_pred = clf.predict(X_test)

### 3. evaluate

In [5]:
accuracy_score(y_test, y_pred)

0.9635500037147353

In [6]:
roc_auc_score(y_test, y_pred)

np.float64(0.5)

In [7]:
roc_curve(y_test, y_pred)

(array([0., 1.]), array([0., 1.]), array([inf,  0.]))